Model Evaluation on Test Data

# Introduction

## Read in libraries, set notebook preferences, and read in data

In [45]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.externals import joblib

In [46]:
#Set options for Pandas
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#Set stle for plots
plt.style.use('ggplot')

In [47]:
#Set path to model location
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\04_Models'

#Read in trained model
model = joblib.load(path + '/2020_0128_Price_Predictor_Final.sav')

#Check model
print('View Model: \n',model)

View Model: 
 GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=-1,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'bootstrap': [False], 

In [48]:
#Read in test data and convert to arrays
test_data = pd.read_csv(path +'/2020_0128_Test_Data.csv').values
test_labels = pd.read_csv(path +'/2020_0128_Test_Labels.csv').values


#Check test data shape
print(test_data.shape)
print(test_labels.shape)

(16442, 82)
(16442, 1)


# Deploy

In [49]:
model.fit(test_data, test_labels)

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:   16.4s remaining:   10.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   21.1s finished
C:\Users\kishe\Miniconda3\lib\site-packages\sklearn\model_selection\_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=-1,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'bootstrap': [False], 'max_depth': [

In [50]:
#View model Metrics
print('Metrics: \nr2: ',np.mean(model.cv_results_['mean_test_r2']))
print('RMSE: ',np.mean(np.sqrt(-1 * (model.cv_results_['mean_test_neg_mean_squared_error']))))
print('MAE: ',np.mean(model.cv_results_['mean_test_neg_mean_absolute_error']))

Metrics: 
r2:  0.9034276275720847
RMSE:  24.34752097226059
MAE:  -13.316888370275406


In [51]:
def feature_importance_viz(cols, ft_importances, feat_count, figsize):
    df = pd.DataFrame(list(zip(cols,ft_importances)), columns = ['Feature', 'Relative_Importance'])
    top50 = df.sort_values(by = 'Relative_Importance', ascending = False).head(feat_count)
    ax = top50.plot(kind = 'barh', x='Feature', figsize=figsize,color = 'green') 
    ax.set_title('Top {} Relative Feature Importances'.format(feat_count))
    ax.invert_yaxis()